In [1]:
import pandas as pd
import numpy as np
import scipy
import scipy.stats
import sklearn
import copy

## Load Data

In [2]:
df_train = pd.read_csv("train.csv", sep=",")

In [3]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df_test = pd.read_csv("test.csv", sep=",")

In [5]:
df = pd.concat([df_train.drop(columns=["Survived"]), df_test], ignore_index=True)

In [6]:
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Data Analysis

In [7]:
df.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,655.000000,2.294882,29.881138,0.498854,0.385027,33.295479
std,378.020061,0.837836,14.413493,1.041658,0.865560,51.758668
min,1.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,328.000000,2.000000,21.000000,0.000000,0.000000,7.895800
50%,655.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,982.000000,3.000000,39.000000,1.000000,0.000000,31.275000
max,1309.000000,3.000000,80.000000,8.000000,9.000000,512.329200


In [8]:
for column in df.columns:
    print(f"{column}: {df[column].isna().sum()/len(df)*100.:3f}%")
print()

PassengerId: 0.000000%
Pclass: 0.000000%
Name: 0.000000%
Sex: 0.000000%
Age: 20.091673%
SibSp: 0.000000%
Parch: 0.000000%
Ticket: 0.000000%
Fare: 0.076394%
Cabin: 77.463713%
Embarked: 0.152788%



In [9]:
df.isna().sum() / len(df) * 100

PassengerId     0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            20.091673
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.076394
Cabin          77.463713
Embarked        0.152788
dtype: float64

In [10]:
df["Cabin"].describe()

count             295
unique            186
top       C23 C25 C27
freq                6
Name: Cabin, dtype: object

In [11]:
df_cabin = df[~df["Cabin"].isna()]

In [12]:
len(df_cabin)

295

In [13]:
df_cabin["Cabin"]

1        C85
3       C123
6        E46
10        G6
11      C103
        ... 
1295     D40
1296     D38
1298     C80
1302     C78
1305    C105
Name: Cabin, Length: 295, dtype: object

### Cabin

In [14]:
df["Cabin"].fillna("N0", inplace=True)

In [15]:
df[df["Cabin"].str.contains(" ")]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
27,28,1,"Fortune, Mr. Charles Alexander",male,19.00,3,2,19950,263.0000,C23 C25 C27,S
75,76,3,"Moen, Mr. Sigurd Hansen",male,25.00,0,0,348123,7.6500,F G73,S
88,89,1,"Fortune, Miss. Mabel Helen",female,23.00,3,2,19950,263.0000,C23 C25 C27,S
97,98,1,"Greenfield, Mr. William Bertram",male,23.00,0,1,PC 17759,63.3583,D10 D12,C
118,119,1,"Baxter, Mr. Quigg Edmond",male,24.00,0,1,PC 17558,247.5208,B58 B60,C
128,129,3,"Peter, Miss. Anna",female,NaN,1,1,2668,22.3583,F E69,C
297,298,1,"Allison, Miss. Helen Loraine",female,2.00,1,2,113781,151.5500,C22 C26,S
299,300,1,"Baxter, Mrs. James (Helene DeLaudeniere Chaput)",female,50.00,0,1,PC 17558,247.5208,B58 B60,C
305,306,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S
311,312,1,"Ryerson, Miss. Emily Borie",female,18.00,2,2,PC 17608,262.3750,B57 B59 B63 B66,C


In [16]:
# kiling lines with 1+ cabins as last and killing lines with letter + cabin as cabin
df["Cabin"] = list(map(lambda x: x[-1], df["Cabin"].str.split(" ").tolist()))

In [17]:
# killing lines where cabin = letter
df["Cabin"] = np.where(df["Cabin"].str.len() < 2, df["Cabin"]+"0", df["Cabin"])

In [18]:
#two alternative cabin
df["Deck"] = df["Cabin"].str[0]
df["CabinN"] = df["Cabin"].str[1:].astype("int")

In [19]:
df["CabinN"]

0         0
1        85
2         0
3       123
4         0
       ... 
1304      0
1305    105
1306      0
1307      0
1308      0
Name: CabinN, Length: 1309, dtype: int32

### Age

In [20]:
# killing nan age values with random dampling with average age and const variance
men_avg_age = df[df["Sex"] == "male"]["Age"].mean() #count magic cinst
wom_avg_age = df[df["Sex"] == "female"]["Age"].mean()
men_avg_age, wom_avg_age = 30.57304863582444, 28.68708762886598 #magic consts on train+test
men_nan_len = len(df[(df["Sex"] == "male") & (df["Age"].isna())])
wom_nan_len = len(df[(df["Sex"] == "female") & (df["Age"].isna())])
missed_mans = np.round(scipy.stats.norm(men_avg_age, 5).rvs(men_nan_len))
missed_woms  = np.round(scipy.stats.norm(wom_avg_age, 5).rvs(wom_nan_len))

In [21]:
df.loc[(df["Sex"]=='male') & (df["Age"].isna()),"Age"] = missed_mans
df.loc[(df["Sex"]=='female') & (df["Age"].isna()),"Age"] = missed_woms

### Fare & Embarked

In [22]:
# too small amount of values to have more clever way to fill it
df["Fare"].fillna(df["Fare"].mean(), inplace=True)
most_fr = df['Embarked'].value_counts().idxmax()
df["Embarked"].fillna(most_fr, inplace=True)

In [23]:
# sanity check
df.isna().sum().sum()

0

### Name

In [24]:
df["Name"].str.extract(r".*\,([^\.]*)\..*")[0].value_counts()

 Mr              757
 Miss            260
 Mrs             197
 Master           61
 Dr                8
 Rev               8
 Col               4
 Major             2
 Mlle              2
 Ms                2
 the Countess      1
 Mme               1
 Capt              1
 Sir               1
 Jonkheer          1
 Don               1
 Lady              1
 Dona              1
Name: 0, dtype: int64

In [25]:
#only pop check from name 
df["IsMiss"] = df["Name"].str.contains(", Miss.", case=1, na=0)
df["IsMr"] = df["Name"].str.contains(", Mr.", case=1, na=0)
df["IsMrs"] = df["Name"].str.contains(", Mrs.", case=1, na=0)

In [26]:
df["IsMiss"]

0       False
1       False
2        True
3       False
4       False
        ...  
1304    False
1305    False
1306    False
1307    False
1308    False
Name: IsMiss, Length: 1309, dtype: bool

### Res df

In [27]:
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Deck,CabinN,IsMiss,IsMr,IsMrs
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,N0,S,N,0,False,True,False
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,C,85,False,True,True
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,N0,S,N,0,True,False,False
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,C,123,False,True,True
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,N0,S,N,0,False,True,False


In [28]:
df["Sex"] = df["Sex"] == "male"

In [29]:
emb_df = pd.get_dummies(df["Embarked"], prefix="embarked")

In [30]:
deck_df = pd.get_dummies(df["Deck"], prefix="deck")

In [31]:
df = pd.concat([df, emb_df, deck_df], axis=1)

In [32]:
df.drop(columns=["Cabin", "Name", "PassengerId", "Ticket", "Embarked", "Deck"], inplace=True)

In [33]:
df = df.astype("float")

In [34]:
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,CabinN,IsMiss,IsMr,IsMrs,...,embarked_S,deck_A,deck_B,deck_C,deck_D,deck_E,deck_F,deck_G,deck_N,deck_T
0,3.0,1.0,22.0,1.0,0.0,7.2500,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,38.0,1.0,0.0,71.2833,85.0,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,0.0,26.0,0.0,0.0,7.9250,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,35.0,1.0,0.0,53.1000,123.0,0.0,1.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,1.0,35.0,0.0,0.0,8.0500,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## Normalization

In [35]:
df.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'CabinN', 'IsMiss',
       'IsMr', 'IsMrs', 'embarked_C', 'embarked_Q', 'embarked_S', 'deck_A',
       'deck_B', 'deck_C', 'deck_D', 'deck_E', 'deck_F', 'deck_G', 'deck_N',
       'deck_T'],
      dtype='object')

In [36]:
min_max_d = {"Pclass": (1, 3), 
             "Age": (0.17, 80.0),
             "Fare": (0.0, 512.3292),
             "CabinN": (0.0, 148.0),
             "SibSp": (0, 8),
             "Parch": (0, 6)}

In [37]:
for i in min_max_d:
    df[i] = (df[i] - min_max_d[i][0]) / (min_max_d[i][1] - min_max_d[i][0])

In [38]:
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,CabinN,IsMiss,IsMr,IsMrs,...,embarked_S,deck_A,deck_B,deck_C,deck_D,deck_E,deck_F,deck_G,deck_N,deck_T
0,1.0,1.0,0.273456,0.125,0.0,0.014151,0.000000,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.473882,0.125,0.0,0.139136,0.574324,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.323563,0.000,0.0,0.015469,0.000000,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.436302,0.125,0.0,0.103644,0.831081,0.0,1.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,0.436302,0.000,0.0,0.015713,0.000000,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## Res Fun

In [45]:
def prepare_all(_df):
    df = copy.deepcopy(_df)
    df["Cabin"].fillna("N0", inplace=True)
    # kiling lines with 1+ cabins as last and killing lines with letter + cabin as cabin
    df["Cabin"] = list(map(lambda x: x[-1], df["Cabin"].str.split(" ").tolist()))
    # killing lines where cabin = letter
    df["Cabin"] = np.where(df["Cabin"].str.len() < 2, df["Cabin"]+"0", df["Cabin"])
    #two alternative cabin
    df["Deck"] = df["Cabin"].str[0]
    df["CabinN"] = df["Cabin"].str[1:].astype("int")
    # killing nan age values with random dampling with average age and const variance
    men_avg_age, wom_avg_age = 30.57304863582444, 28.68708762886598 #magic consts on train+test
    men_nan_len = len(df[(df["Sex"] == "male") & (df["Age"].isna())])
    wom_nan_len = len(df[(df["Sex"] == "female") & (df["Age"].isna())])
    missed_mans = np.round(scipy.stats.norm(men_avg_age, 5).rvs(men_nan_len))
    missed_woms  = np.round(scipy.stats.norm(wom_avg_age, 5).rvs(wom_nan_len))
    df.loc[(df["Sex"]=='male') & (df["Age"].isna()),"Age"] = missed_mans
    df.loc[(df["Sex"]=='female') & (df["Age"].isna()),"Age"] = missed_woms
    # too small amount of values to have more clever way to fill it
    fare_mean_c = 33.29547928134557
    df["Fare"].fillna(fare_mean_c, inplace=True)
    most_fr = "S"
    df["Embarked"].fillna(most_fr, inplace=True)
    #only pop check from name 
    df["IsMiss"] = df["Name"].str.contains(", Miss.", case=1, na=0)
    df["IsMr"] = df["Name"].str.contains(", Mr.", case=1, na=0)
    df["IsMrs"] = df["Name"].str.contains(", Mrs.", case=1, na=0)
    df["Sex"] = df["Sex"] == "male"
    emb_df = pd.get_dummies(df["Embarked"], prefix="embarked")
    deck_df = pd.get_dummies(df["Deck"], prefix="deck")
    df = pd.concat([df, emb_df, deck_df], axis=1)
    df.drop(columns=["Cabin", "Name", "PassengerId", "Ticket", "Embarked", "Deck"], inplace=True)
    df = df.astype("float")
    min_max_d = {"Pclass": (1, 3), 
             "Age": (0.17, 80.0),
             "Fare": (0.0, 512.3292),
             "CabinN": (0.0, 148.0),
             "SibSp": (0, 8),
             "Parch": (0, 6)}
    for i in min_max_d:
        df[i] = (df[i] - min_max_d[i][0]) / (min_max_d[i][1] - min_max_d[i][0])
    for i in set(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'CabinN', 'IsMiss',
       'IsMr', 'IsMrs', 'embarked_C', 'embarked_Q', 'embarked_S', 'deck_A',
       'deck_B', 'deck_C', 'deck_D', 'deck_E', 'deck_F', 'deck_G', 'deck_N',
       'deck_T']) - set(df.columns):
        df[i] = [0 for i in range(len(df))]
    
    return df

In [46]:
df_train = pd.read_csv("train.csv", sep=",")
df_train = prepare_all(df_train)
df_train.to_csv("train_preprocessed.csv", index=False)

In [47]:
df_test = pd.read_csv("test.csv", sep=",")
df_test = prepare_all(df_test)
df_test.to_csv("test_preprocessed.csv", index=False)